In [1]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import wandb
from util import load_data
from TimeGAN import TimeGAN
from train import embedding_trainer, supervisor_trainer, joint_trainer
from dataset import TimeGANDataset

params = {
    'Z_dim': 50,
    'hidden_dim': 64,
    'batch_size': 10,
    'dis_thresh': 0.15,
    'num_layers': 5,
    'embedder_epoch': 2,
    'supervisor_epochs': 2,
    'padding_value': -1,
    'module': 'GRU',
    'device': 'cpu',
    'data_source': '12var'
}

In [2]:
data=load_data('../Data/PreProcessed/29var/df29.xlsx')

Number of unique Names: 187
Number of columns: 30
Number of rows: 31257
Name of columns: Index(['Name', 'Sad', 'Guilty', 'Happy', 'Hopeless', 'Anxious', 'Stressed',
       'Overwhelmed', 'Angry', 'Calm', 'Energetic', 'Lonely', 'In pain',
       'Dizzy', 'Nauseous', 'Racing Heart', 'Look forward completing tasks',
       'Satisfied with myself', 'Satisfied with my physical appearance',
       'Cravings', 'Crave_Food', 'Enjoying moment', 'With how many people',
       'Eating', 'Eating_healthy', 'In control', 'Concentrated', 'Worried',
       'WrongDoing', 'Impulsivity'],
      dtype='object')
Number of samples: 180
Number of Names: 187
Shape of np_samples: (180, 120, 29)


In [3]:
sequence_length = data.shape[1]
print('Parameter Sequence Length:',sequence_length)
number_of_features = data.shape[2]
print('Parameter Number of Features:',number_of_features)
Y_temporal=[len(x) for x in data]
params['input_dim'] = number_of_features
params['max_seq_len'] = sequence_length

Parameter Sequence Length: 120
Parameter Number of Features: 29


In [4]:
data.shape

(180, 120, 29)

In [5]:
model=TimeGAN(params)

In [6]:
dataset=TimeGANDataset(data,Y_temporal)
dataloader=torch.utils.data.DataLoader(dataset,batch_size=params['batch_size'],shuffle=True)

In [7]:
for i, data in enumerate(dataloader):
    break

In [8]:
data[0].shape

torch.Size([10, 120, 29])

In [9]:
model.embedder(data[0],data[1]).shape

torch.Size([10, 120, 64])

In [10]:
Z = torch.randn(32, 200, 50)  # (batch_size, seq_len, Z_dim)

In [11]:
model.generator(Z).shape

torch.Size([32, 200, 64])

In [17]:
model.discriminator(model.embedder(data[0],data[1])).shape

torch.Size([10, 120])

In [20]:
model.recovery(model.embedder(data[0],data[1])).shape

torch.Size([10, 120, 29])

In [22]:
model.forward(X=None, T=params['max_seq_len'],Z=Z, obj='inference').shape

torch.Size([32, 200, 29])